In [26]:
import pandas as pd
import numpy as np
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
from dustmaps.sfd import SFDQuery
from dustmaps.sfd import SFDWebQuery
from astroquery.vizier import Vizier
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"

import matplotlib
import matplotlib.pyplot as plt
label_size = 24
matplotlib.rc('font', size=label_size) 

In [16]:
# Run only once
# import dustmaps.sfd
# dustmaps.sfd.fetch()

 64.0 MiB of 64.0 MiB |   6.2 MiB/s |###################| 100% | ETA:  00:00:00

 64.0 MiB of 64.0 MiB |  10.4 MiB/s |###################| 100% | ETA:  00:00:00

In [11]:
# import data
ste_par_all = pd.read_csv("/Users/xwou/Dropbox (MIT)/atari_abundances_Mohammad/data/SMSS88_loop0_w_feh.csv", usecols=['Name','RA','DEC','teff_ini','logg_ini','feh_ini'])


In [13]:
# Rename the columns to match what the script wants
ste_par_all = ste_par_all.rename(columns={"Name": "name", "RA": "ra", "DEC": "dec", "teff_ini": "teff", "logg_ini": "logg", "feh_ini": "mh"})
# Create place holder columns for estat
ste_par_all['estat_teff'] = 100.
ste_par_all['estat_logg'] = 0.15
ste_par_all['estat_mh'] = 0.2
ste_par_all['estat_vt'] = 0.15
ste_par_all['vt'] = 1.6

In [14]:
ste_par_all

,name,ra,dec,teff,logg,mh,estat_teff,estat_logg,estat_mh,estat_vt,vt
0,SMSS5,82.389850,-7.399917,4648,1.20,-2.41,100.0,0.15,0.2,0.15,1.6
1,SMSS6,84.340167,-13.061573,5156,1.95,-1.35,100.0,0.15,0.2,0.15,1.6
2,SMSS9,88.329642,-54.266350,4927,2.00,-2.07,100.0,0.15,0.2,0.15,1.6
3,SMSS11,89.861633,-33.773320,4927,1.55,-1.78,100.0,0.15,0.2,0.15,1.6
4,SMSS14,151.745996,-5.898819,5260,2.95,-1.52,100.0,0.15,0.2,0.15,1.6
...,...,...,...,...,...,...,...,...,...,...,...
83,SMSS735,142.907786,-5.196365,4450,0.70,-2.75,100.0,0.15,0.2,0.15,1.6
84,SMSS747,346.204727,-42.730013,4450,0.50,-3.44,100.0,0.15,0.2,0.15,1.6
85,SMSS753,352.581743,-8.220890,4540,0.65,-3.24,100.0,0.15,0.2,0.15,1.6
86,SMSS757,355.315969,-64.112271,4765,1.00,-2.70,100.0,0.15,0.2,0.15,1.6


In [40]:
# Query Vizier for BJ21 distances
# Query dustmaps for SF11 E(B-V)
sfd = SFDQuery()
col_gaia3 = ['source_id','l','b','parallax','parallax_error','phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag',\
             'nu_eff_used_in_astrometry','pseudocolour','ecl_lat','astrometric_params_solved','ruwe']
for i in range(len(ste_par_all)):
    # Query Vizier for BJ21 distances
    ra_tmp = ste_par_all['ra'][i]
    dec_tmp = ste_par_all['dec'][i]
    coords = SkyCoord(ra=ra_tmp, dec=dec_tmp,unit=(u.deg, u.deg),frame='icrs')
    result = Vizier.query_region(coords,
                            radius=5*u.arcsecond,
                            catalog="I/352") # "I/352" is the catalog for BJ21
    ste_par_all.loc[i,'dist'] = result[0]['rpgeo'][0]
    ste_par_all.loc[i,'e_dist'] = (result[0]['B_rpgeo'][0] - result[0]['b_rpgeo'][0])/2
    
    # Query Gaia for Gaia EDR3 
    j = Gaia.cone_search_async(coords, radius=u.Quantity(5.0, u.arcsecond))
    r = j.get_results()
    for j in range(len(col_gaia3)):
        ste_par_all.loc[i,col_gaia3[j]] = r[col_gaia3[j]][0]
        
# Query dustmaps for SF11 E(B-V)
ebv = sfd(SkyCoord(ra=ste_par_all['ra'], dec=ste_par_all['dec'],unit=(u.deg, u.deg),frame='icrs'))
e_ebv = 0.0001
ste_par_all['ebv'] = ebv
ste_par_all['e_ebv'] = e_ebv

INFO: Query finished. [astroquery.utils.tap.core]


/opt/anaconda3/envs/pandas_project/lib/python3.8/site-packages/pandas/core/dtypes/missing.py:550: UserWarning: Warning: converting a masked element to nan.
  val = np.array(val, copy=False)
/opt/anaconda3/envs/pandas_project/lib/python3.8/site-packages/pandas/core/internals/blocks.py:953: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = value


INFO: Query finished. [astroquery.utils.tap.core]


/opt/anaconda3/envs/pandas_project/lib/python3.8/site-packages/pandas/core/internals/blocks.py:953: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = value


INFO: Query finished. [astroquery.utils.tap.core]


...

/opt/anaconda3/envs/pandas_project/lib/python3.8/site-packages/pandas/core/internals/blocks.py:953: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = value


INFO: Query finished. [astroquery.utils.tap.core]


/opt/anaconda3/envs/pandas_project/lib/python3.8/site-packages/pandas/core/internals/blocks.py:953: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = value


INFO: Query finished. [astroquery.utils.tap.core]


/opt/anaconda3/envs/pandas_project/lib/python3.8/site-packages/pandas/core/internals/blocks.py:953: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = value


In [41]:
ste_par_all

,name,ra,dec,teff,logg,mh,estat_teff,estat_logg,estat_mh,estat_vt,...,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,nu_eff_used_in_astrometry,pseudocolour,ecl_lat,astrometric_params_solved,ruwe,ebv,e_ebv
0,SMSS5,82.389850,-7.399917,4648,1.20,-2.41,100.0,0.15,0.2,0.15,...,11.138597,11.729438,10.396433,1.450518,NaN,-30.607650,31.0,0.975602,0.139826,0.0001
1,SMSS6,84.340167,-13.061573,5156,1.95,-1.35,100.0,0.15,0.2,0.15,...,13.399730,13.865712,12.759246,1.494794,NaN,-36.366332,31.0,1.021162,0.153754,0.0001
2,SMSS9,88.329642,-54.266350,4927,2.00,-2.07,100.0,0.15,0.2,0.15,...,13.714207,14.196639,13.059258,1.489456,NaN,-77.679083,31.0,1.061873,0.098747,0.0001
3,SMSS11,89.861633,-33.773320,4927,1.55,-1.78,100.0,0.15,0.2,0.15,...,11.483175,11.940176,10.851027,1.498688,NaN,-57.212486,31.0,0.879778,0.038642,0.0001
4,SMSS14,151.745996,-5.898819,5260,2.95,-1.52,100.0,0.15,0.2,0.15,...,14.516273,14.910448,13.945849,1.525681,NaN,-16.355437,31.0,1.024084,0.045209,0.0001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,SMSS735,142.907786,-5.196365,4450,0.70,-2.75,100.0,0.15,0.2,0.15,...,13.036031,13.686154,12.257299,1.431894,NaN,-18.784491,31.0,1.084293,0.036989,0.0001
84,SMSS747,346.204727,-42.730013,4450,0.50,-3.44,100.0,0.15,0.2,0.15,...,11.829420,12.411304,11.098705,1.454288,NaN,-33.564748,31.0,0.983345,0.007934,0.0001
85,SMSS753,352.581743,-8.220890,4540,0.65,-3.24,100.0,0.15,0.2,0.15,...,11.819242,12.339817,11.133599,1.474877,NaN,-4.609323,31.0,0.773171,0.028858,0.0001
86,SMSS757,355.315969,-64.112271,4765,1.00,-2.70,100.0,0.15,0.2,0.15,...,12.544172,13.015709,11.897929,1.493817,NaN,-54.216470,31.0,1.047507,0.021165,0.0001


In [29]:
ra_tmp = ste_par_all['ra'][1]
dec_tmp = ste_par_all['dec'][1]
coords = SkyCoord(ra=ra_tmp, dec=dec_tmp,unit=(u.deg, u.deg),frame='icrs')

In [31]:
coords = SkyCoord(ra_tmp, dec_tmp, unit=(u.deg, u.deg),frame='icrs')
sfd = SFDQuery()
ebv = sfd(coords)

print('E(B-V) = {:.5f} mag'.format(ebv*0.86))

# sfdweb = SFDWebQuery()
# ebv = sfdweb(coords)

# print('E(B-V) = {:.5f} mag'.format(ebv*0.86))

E(B-V) = 0.13223 mag


In [32]:
j = Gaia.cone_search_async(coords, radius=u.Quantity(5.0, u.arcsecond))
r = j.get_results()
print(r)

INFO: Query finished. [astroquery.utils.tap.core]
    solution_id             DESIGNATION               source_id      random_index ... ebpminrp_gspphot_lower ebpminrp_gspphot_upper libname_gspphot          dist         
                                                                                  ...          mag                    mag                                                 
------------------- ---------------------------- ------------------- ------------ ... ---------------------- ---------------------- --------------- ----------------------
1636148068921376768 Gaia DR3 3008833708513620352 3008833708513620352    294383468 ...                 0.4322                 0.4369           MARCS 1.1063966700041202e-05


In [36]:
r['b'][0]

-22.235833944658008

In [55]:
# geall_gaia3.loc[:,['ra','dec']].to_csv('/Users/xwou/Dropbox (MIT)/GSE_abundances/literature_data/geall_coord.csv',index=False)

In [15]:
col_gaia3 = ['source_id','ra','dec','l','b','parallax','parallax_error','phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag',\
             'nu_eff_used_in_astrometry','pseudocolour','ecl_lat','astrometric_params_solved','ruwe']
col_ebv = ['mean_E_B_V_SandF','stdev_E_B_V_SandF']
col_sp = ['name','teff','estat_teff','logg','estat_logg','vt','estat_vt','mh','estat_mh']

In [39]:
# df_outmr = gse_summary_mr.loc[:,col_sp].merge(gemr_ebv.loc[:,col_ebv],left_index=True,right_index=True)
# df_outmr = df_outmr.merge(gemr_gaia3.loc[:,col_gaia3],left_index=True,right_index=True)

In [40]:
# df_outmp = gse_summary_mp.loc[:,col_sp].merge(gemp_ebv.loc[:,col_ebv],left_index=True,right_index=True)
# df_outmp = df_outmp.merge(gemp_gaia3.loc[:,col_gaia3],left_index=True,right_index=True)

In [16]:
df_outall = gse_summary_all.loc[:,col_sp].merge(geall_ebv.loc[:,col_ebv],left_index=True,right_index=True)
df_outall = df_outall.merge(geall_gaia3.loc[:,col_gaia3],left_index=True,right_index=True)

In [17]:
# rename some columns
# df_outmr = df_outmr.rename(columns={'mean_E_B_V_SandF': 'ebv', 'stdev_E_B_V_SandF': 'e_ebv'})
# df_outmp = df_outmp.rename(columns={'mean_E_B_V_SandF': 'ebv', 'stdev_E_B_V_SandF': 'e_ebv'})
df_outall = df_outall.rename(columns={'mean_E_B_V_SandF': 'ebv', 'stdev_E_B_V_SandF': 'e_ebv'})

df_outall['dist'], df_outall['e_dist'] = geall_bj21['rpgeo'], (geall_bj21['B_rpgeo']-geall_bj21['b_rpgeo'])/2

In [42]:
# df_outmr

,name,teff,estat_teff,logg,estat_logg,vt,estat_vt,mh,estat_mh,ebv,...,source_id,parallax,parallax_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,nu_eff_used_in_astrometry,pseudocolour,ecl_lat,astrometric_params_solved
0,2m00400030+0101367,4972.0,74,2.38,0.096687,1.68,0.127721,-1.42,0.14,0.0176,...,2543390232947773824,0.240315,0.016789,12.822310,13.286703,12.191200,1.496747,NaN,-3.016856,31
1,2m01072703+0119276,4945.0,93,2.62,0.122813,1.62,0.125126,-1.12,0.39,0.0196,...,2538202737087917184,0.401362,0.019872,12.798253,13.278104,12.153212,1.489578,NaN,-5.402542,31
2,2m01301391+0207076,4990.0,62,1.85,0.070098,1.80,0.149962,-1.20,0.12,0.0160,...,2558587476508131072,0.133845,0.016505,13.288914,13.758636,12.649494,1.492299,NaN,-6.809573,31
3,2m01562961-0150460,4585.0,72,1.72,0.096750,1.80,0.115349,-1.28,0.12,0.0233,...,2505065101598396416,0.196824,0.020415,11.680469,12.297540,10.926215,1.441752,NaN,-12.888316,31
4,2m01593915-0242496,4756.0,59,1.71,0.074813,1.92,0.111802,-1.56,0.10,0.0243,...,2504634406573540608,0.441742,0.020831,11.464257,11.972621,10.790941,1.478006,NaN,-13.980093,31
5,2m14231377-0520165,4315.0,432,1.77,0.082969,1.25,0.175190,-0.41,0.24,0.0427,...,3642334203486585728,0.069779,0.019296,12.216512,12.965445,11.375617,NaN,1.405987,8.416144,95
6,2m14530873+1611041,4585.0,115,1.82,0.096687,1.68,0.123100,-0.92,0.17,0.0190,...,1187898485610541696,0.263752,0.014535,10.872484,11.501062,10.109803,1.437486,NaN,31.175095,31
7,2m15185770+0134483,4423.0,117,1.81,0.101813,1.70,0.149202,-0.92,0.61,0.0409,...,4420695900880776576,0.104583,0.013642,12.130324,12.843897,11.312766,1.411687,NaN,19.192109,31
8,2m20551529-0054163,4414.0,117,1.84,0.092000,1.73,0.151102,-0.89,0.24,0.0831,...,4227203402244069120,0.078485,0.014357,12.705564,13.482205,11.848728,1.394432,NaN,15.813970,31
9,2m23152484+1216037,4720.0,74,2.15,0.114219,1.18,0.158156,-0.82,0.06,0.0521,...,2811948250134539520,0.121536,0.021093,13.454059,14.039059,12.723642,1.451205,NaN,15.669480,31


In [45]:
# df_outmr.to_csv('/Users/xwou/Dropbox (MIT)/GSE_abundances/photo_ste_par_gemr_input.csv',index=False)

In [44]:
# df_outmp

,name,teff,estat_teff,logg,estat_logg,vt,estat_vt,mh,estat_mh,ebv,...,source_id,parallax,parallax_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,nu_eff_used_in_astrometry,pseudocolour,ecl_lat,astrometric_params_solved
0,hd122563,4567.0,18,0.86,0.072563,1.98,0.131559,-2.77,0.09,0.0218,...,3723554268436602240,3.099057,0.033201,5.874960,6.397473,5.178669,1.479162,NaN,20.741221,31
1,ge01,4495.0,105,1.98,0.080438,1.42,0.125452,-1.01,0.24,0.1990,...,4377687545785813248,0.170254,0.011691,12.622972,13.412316,11.753124,1.390560,NaN,19.002220,31
2,ge02,4450.0,158,2.01,0.081656,1.48,0.159230,-0.82,0.36,0.3399,...,6243816407441434752,0.107002,0.018491,13.057759,13.915890,12.146782,1.375633,NaN,0.072409,31
3,ge03,4450.0,153,1.90,0.074961,1.51,0.155719,-0.67,0.80,0.0992,...,4081038754562022528,0.036186,0.014992,13.596936,14.366990,12.741849,1.397952,NaN,0.132229,31
4,ge-04,4720.0,70,2.21,0.103594,1.12,0.167561,-0.88,0.04,0.2037,...,4045622011959560832,1.402434,0.742940,19.543488,18.735480,17.088512,NaN,1.565775,-8.941845,95
5,2massj21201351-3354026,4630.0,36,1.08,0.077625,1.99,0.082266,-2.64,0.07,0.0813,...,6782821815106683008,0.277055,0.015451,12.241519,12.781894,11.540783,1.468713,NaN,-17.443215,31
6,2massj05455248-2556383,4873.0,49,1.83,0.102938,1.90,0.087382,-2.46,0.08,0.0217,...,2914192523598690048,0.380170,0.011316,12.655770,13.073313,12.060354,1.516442,NaN,-49.323550,31
7,119585916,4648.0,33,0.93,0.127875,1.90,0.135740,-2.27,0.09,0.0575,...,2659914516754259456,0.048183,0.041338,15.111205,15.650965,14.401452,1.466567,NaN,7.313242,31
8,117663868,4540.0,39,1.15,0.086250,2.16,0.148865,-2.16,0.10,0.0646,...,2792504722733373312,2.108116,0.623453,20.328655,21.207245,19.177391,NaN,1.288861,12.218525,95
9,113719678,5080.0,119,2.72,0.119000,1.50,0.136757,-2.55,0.19,0.0710,...,2733469542550276352,0.131934,0.043760,15.568827,15.974277,14.974508,1.519009,NaN,22.445934,31


In [46]:
# df_outmp.to_csv('/Users/xwou/Dropbox (MIT)/GSE_abundances/photo_ste_par_gemp_input.csv',index=False)

In [8]:
gse_summary_all

,name,teff,logg,vt,mh,estat_teff,estat_logg,estat_vt,estat_mh,esys_teff,esys_logg,esys_vt,esys_mh,etot_teff,etot_logg,etot_vt,etot_mh
0,2m00400030+0101367,4900.0,2.29,1.54,-1.42,38,0.050094,0.120546,0.07,100,0.2,0.2,0.2,106.976633,0.206178,0.233519,0.211896
1,2m01072703+0119276,4945.0,2.63,1.13,-0.83,54,0.049313,0.128186,0.11,100,0.2,0.2,0.2,113.648581,0.205990,0.237553,0.228254
2,2m01301391+0207076,4945.0,1.74,1.88,-1.30,42,0.059812,0.134551,0.07,100,0.2,0.2,0.2,108.461975,0.208752,0.241048,0.211896
3,2m01562961-0150460,4585.0,1.81,1.80,-1.25,47,0.036766,0.121098,0.07,100,0.2,0.2,0.2,110.494344,0.203351,0.233805,0.211896
4,2m01593915-0242496,4765.0,1.96,1.68,-1.38,45,0.073500,0.140478,0.07,100,0.2,0.2,0.2,109.658561,0.213078,0.244406,0.211896
5,2m14231377-0520165,4315.0,1.73,1.50,-0.78,98,0.108125,0.181573,0.32,100,0.2,0.2,0.2,140.014285,0.227357,0.270127,0.377359
6,2m14530873+1611041,4540.0,1.71,1.79,-1.11,60,0.053438,0.124880,0.08,100,0.2,0.2,0.2,116.619038,0.207016,0.235786,0.215407
7,2m15185770+0134483,4369.0,1.67,1.80,-1.16,68,0.084805,0.147734,0.11,100,0.2,0.2,0.2,120.929732,0.217237,0.248647,0.228254
8,2m20551529-0054163,4369.0,1.85,1.63,-0.96,75,0.080938,0.154032,0.14,100,0.2,0.2,0.2,125.000000,0.215757,0.252440,0.244131
9,2m23152484+1216037,4675.0,2.07,1.55,-1.08,47,0.051750,0.131074,0.07,100,0.2,0.2,0.2,110.494344,0.206587,0.239124,0.211896


In [18]:
df_outall

,name,teff,estat_teff,logg,estat_logg,vt,estat_vt,mh,estat_mh,ebv,...,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,nu_eff_used_in_astrometry,pseudocolour,ecl_lat,astrometric_params_solved,ruwe,dist,e_dist
0,2m00400030+0101367,4900.0,38,2.29,0.050094,1.54,0.120546,-1.42,0.07,0.0176,...,12.822310,13.286703,12.191200,1.496747,NaN,-3.016856,31,0.956245,4091.224850,274.214965
1,2m01072703+0119276,4945.0,54,2.63,0.049313,1.13,0.128186,-0.83,0.11,0.0196,...,12.798253,13.278104,12.153212,1.489578,NaN,-5.402542,31,1.031802,2401.767820,113.949830
2,2m01301391+0207076,4945.0,42,1.74,0.059812,1.88,0.134551,-1.30,0.07,0.0160,...,13.288914,13.758636,12.649494,1.492299,NaN,-6.809573,31,1.022474,5363.537600,720.047850
3,2m01562961-0150460,4585.0,47,1.81,0.036766,1.80,0.121098,-1.25,0.07,0.0233,...,11.680469,12.297540,10.926215,1.441752,NaN,-12.888316,31,1.079671,4277.297360,328.687500
4,2m01593915-0242496,4765.0,45,1.96,0.073500,1.68,0.140478,-1.38,0.07,0.0243,...,11.464257,11.972621,10.790941,1.478006,NaN,-13.980093,31,0.991366,2265.075930,169.502500
5,2m14231377-0520165,4315.0,98,1.73,0.108125,1.50,0.181573,-0.78,0.32,0.0427,...,12.216512,12.965445,11.375617,NaN,1.405987,8.416144,95,1.162812,15783.599600,3983.757800
6,2m14530873+1611041,4540.0,60,1.71,0.053438,1.79,0.124880,-1.11,0.08,0.0190,...,10.872484,11.501062,10.109803,1.437486,NaN,31.175095,31,0.964604,3397.374760,152.630375
7,2m15185770+0134483,4369.0,68,1.67,0.084805,1.80,0.147734,-1.16,0.11,0.0409,...,12.130324,12.843897,11.312766,1.411687,NaN,19.192109,31,1.109457,8502.754880,1166.510010
8,2m20551529-0054163,4369.0,75,1.85,0.080938,1.63,0.154032,-0.96,0.14,0.0831,...,12.705564,13.482205,11.848728,1.394432,NaN,15.813970,31,0.964352,10156.610400,985.952165
9,2m23152484+1216037,4675.0,47,2.07,0.051750,1.55,0.131074,-1.08,0.07,0.0521,...,13.454059,14.039059,12.723642,1.451205,NaN,15.669480,31,0.984595,5908.095700,412.169675


In [23]:
gse_abund_all[mask_mg]

,species,N,logeps,stdev,stderr,[X/H],[X/Fe],star
9,12.0,4,6.45,0.13,0.07,-1.15,0.26,2m00400030+0101367
23,12.0,4,6.77,0.21,0.1,-0.83,-0.01,2m01072703+0119276
37,12.0,4,6.65,0.1,0.05,-0.95,0.31,2m01301391+0207076
51,12.0,4,6.6,0.17,0.09,-1.0,0.24,2m01562961-0150460
66,12.0,4,6.56,0.17,0.09,-1.04,0.34,2m01593915-0242496
80,12.0,4,6.85,0.26,0.13,-0.75,-0.01,2m14231377-0520165
95,12.0,4,6.7,0.19,0.1,-0.9,0.21,2m14530873+1611041
110,12.0,4,6.66,0.23,0.12,-0.94,0.21,2m15185770+0134483
125,12.0,4,6.65,0.24,0.12,-0.95,0.02,2m20551529-0054163
140,12.0,4,6.64,0.19,0.1,-0.96,0.13,2m23152484+1216037


In [21]:
# Add alpha measurement to the final table as well
df_outall['alphafe'] = np.nan
df_outall['estat_alphafe'] = np.nan
mask_mg = gse_abund_all['species'] == '12.0'
for i in range(len(df_outall)):
    name_tmp = df_outall['name'][i]
    
    mask_name = gse_abund_all['star'] == name_tmp
    
    i_match = np.where(mask_mg & mask_name)[0][0]
    
    df_outall.loc[i,'alphafe'] = float(gse_abund_all.loc[i_match,'[X/Fe]'])
    df_outall.loc[i,'estat_alphafe'] = float(gse_abund_all.loc[i_match,'stdev'])

In [22]:
df_outall

,name,teff,estat_teff,logg,estat_logg,vt,estat_vt,mh,estat_mh,ebv,...,phot_rp_mean_mag,nu_eff_used_in_astrometry,pseudocolour,ecl_lat,astrometric_params_solved,ruwe,dist,e_dist,alphafe,estat_alphafe
0,2m00400030+0101367,4900.0,38,2.29,0.050094,1.54,0.120546,-1.42,0.07,0.0176,...,12.191200,1.496747,NaN,-3.016856,31,0.956245,4091.224850,274.214965,0.26,0.13
1,2m01072703+0119276,4945.0,54,2.63,0.049313,1.13,0.128186,-0.83,0.11,0.0196,...,12.153212,1.489578,NaN,-5.402542,31,1.031802,2401.767820,113.949830,-0.01,0.21
2,2m01301391+0207076,4945.0,42,1.74,0.059812,1.88,0.134551,-1.30,0.07,0.0160,...,12.649494,1.492299,NaN,-6.809573,31,1.022474,5363.537600,720.047850,0.31,0.10
3,2m01562961-0150460,4585.0,47,1.81,0.036766,1.80,0.121098,-1.25,0.07,0.0233,...,10.926215,1.441752,NaN,-12.888316,31,1.079671,4277.297360,328.687500,0.24,0.17
4,2m01593915-0242496,4765.0,45,1.96,0.073500,1.68,0.140478,-1.38,0.07,0.0243,...,10.790941,1.478006,NaN,-13.980093,31,0.991366,2265.075930,169.502500,0.34,0.17
5,2m14231377-0520165,4315.0,98,1.73,0.108125,1.50,0.181573,-0.78,0.32,0.0427,...,11.375617,NaN,1.405987,8.416144,95,1.162812,15783.599600,3983.757800,-0.01,0.26
6,2m14530873+1611041,4540.0,60,1.71,0.053438,1.79,0.124880,-1.11,0.08,0.0190,...,10.109803,1.437486,NaN,31.175095,31,0.964604,3397.374760,152.630375,0.21,0.19
7,2m15185770+0134483,4369.0,68,1.67,0.084805,1.80,0.147734,-1.16,0.11,0.0409,...,11.312766,1.411687,NaN,19.192109,31,1.109457,8502.754880,1166.510010,0.21,0.23
8,2m20551529-0054163,4369.0,75,1.85,0.080938,1.63,0.154032,-0.96,0.14,0.0831,...,11.848728,1.394432,NaN,15.813970,31,0.964352,10156.610400,985.952165,0.02,0.24
9,2m23152484+1216037,4675.0,47,2.07,0.051750,1.55,0.131074,-1.08,0.07,0.0521,...,12.723642,1.451205,NaN,15.669480,31,0.984595,5908.095700,412.169675,0.13,0.19


In [11]:
df_outall['ruwe'].describe()

count    44.000000
mean      1.014670
std       0.061329
min       0.916422
25%       0.967142
50%       1.017199
75%       1.041533
max       1.162812
Name: ruwe, dtype: float64

In [24]:
# df_outall.to_csv('/Users/xwou/Dropbox (MIT)/GSE_abundances/photo_ste_par_geall_input.csv',index=False)
df_outall.to_csv('/Users/xwou/Dropbox (MIT)/GSE_abundances/photo_ste_par_geall_v5_input.csv',index=False)
df_outall.to_csv('/Users/xwou/Dropbox (MIT)/GSE_abundances/photoiso_ste_par_geall_v5_input.csv',index=False)

In [13]:
gse_summary_mr

,name,teff,logg,vt,mh,estat_teff,estat_logg,estat_vt,estat_mh,esys_teff,esys_logg,esys_vt,esys_mh,etot_teff,etot_logg,etot_vt,etot_mh
0,2m00400030+0101367,4972.0,2.38,1.68,-1.42,74,0.096687,0.127721,0.14,100,0.2,0.2,0.2,124.402572,0.222145,0.237303,0.244131
1,2m01072703+0119276,4945.0,2.62,1.62,-1.12,93,0.122813,0.125126,0.39,100,0.2,0.2,0.2,136.561342,0.234697,0.235916,0.438292
2,2m01301391+0207076,4990.0,1.85,1.80,-1.20,62,0.070098,0.149962,0.12,100,0.2,0.2,0.2,117.660529,0.211928,0.249977,0.233238
3,2m01562961-0150460,4585.0,1.72,1.80,-1.28,72,0.096750,0.115349,0.12,100,0.2,0.2,0.2,123.223374,0.222172,0.230880,0.233238
4,2m01593915-0242496,4756.0,1.71,1.92,-1.56,59,0.074813,0.111802,0.10,100,0.2,0.2,0.2,116.107709,0.213534,0.229128,0.223607
5,2m14231377-0520165,4315.0,1.77,1.25,-0.41,432,0.082969,0.175190,0.24,100,0.2,0.2,0.2,443.423049,0.216527,0.265878,0.312410
6,2m14530873+1611041,4585.0,1.82,1.68,-0.92,115,0.096687,0.123100,0.17,100,0.2,0.2,0.2,152.397507,0.222145,0.234848,0.262488
7,2m15185770+0134483,4423.0,1.81,1.70,-0.92,117,0.101813,0.149202,0.61,100,0.2,0.2,0.2,153.912313,0.224423,0.249522,0.641950
8,2m20551529-0054163,4414.0,1.84,1.73,-0.89,117,0.092000,0.151102,0.24,100,0.2,0.2,0.2,153.912313,0.220145,0.250663,0.312410
9,2m23152484+1216037,4720.0,2.15,1.18,-0.82,74,0.114219,0.158156,0.06,100,0.2,0.2,0.2,124.402572,0.230317,0.254977,0.208806


In [16]:
gemr_ebv

,objname,ra,dec,cutout_size,E_B_V_SandF,mean_E_B_V_SandF,stdev_E_B_V_SandF,max_E_B_V_SandF,min_E_B_V_SandF,AV_SandF,E_B_V_SFD,mean_E_B_V_SFD,stdev_E_B_V_SFD,max_E_B_V_SFD,min_E_B_V_SFD,AV_SFD,errmsg
0,2MASS J00400030+0101367,10.00125,1.02686,5.0,0.0175,0.0176,0.0005,0.0184,0.0168,0.0541,0.0203,0.0205,0.0006,0.0214,0.0195,0.0629,<NA>
1,2MASS J01072703+0119276,16.86262,1.32433,5.0,0.0192,0.0196,0.0009,0.0217,0.0184,0.0596,0.0224,0.0227,0.0011,0.0253,0.0214,0.0693,<NA>
2,2MASS J01301391+0207076,22.55796,2.11878,5.0,0.0157,0.0160,0.0005,0.0172,0.0149,0.0487,0.0182,0.0186,0.0006,0.0201,0.0174,0.0566,<NA>
3,2MASS J01562961-0150460,29.12338,-1.84611,5.0,0.0233,0.0233,0.0005,0.0242,0.0227,0.0721,0.0270,0.0271,0.0005,0.0282,0.0264,0.0838,<NA>
4,2MASS J01593915-0242496,29.91313,-2.71378,5.0,0.0242,0.0243,0.0007,0.0256,0.0232,0.0750,0.0281,0.0282,0.0008,0.0298,0.0269,0.0872,<NA>
5,2MASS J14231377-0520165,215.80738,-5.33792,5.0,0.0434,0.0427,0.0019,0.0462,0.0395,0.1346,0.0505,0.0497,0.0022,0.0537,0.0459,0.1565,<NA>
6,2MASS J14530873+1611041,223.28638,16.18447,5.0,0.0191,0.0190,0.0003,0.0195,0.0183,0.0592,0.0222,0.0221,0.0004,0.0227,0.0213,0.0688,<NA>
7,2MASS J15185770+0134483,229.74042,1.58008,5.0,0.0413,0.0409,0.0006,0.0419,0.0397,0.1280,0.0480,0.0476,0.0007,0.0487,0.0462,0.1489,<NA>
8,2MASS J20551529-0054163,313.81371,-0.90453,5.0,0.0845,0.0831,0.0036,0.0910,0.0792,0.2620,0.0983,0.0966,0.0042,0.1058,0.0921,0.3046,<NA>
9,2MASS J23152484+1216037,348.85350,12.26769,5.0,0.0507,0.0521,0.0024,0.0564,0.0482,0.1572,0.0590,0.0606,0.0028,0.0656,0.0560,0.1828,<NA>


In [49]:
gemp_gaia3

,input_position,distance,source_id,designation,ra,dec,l,b,ecl_lon,ecl_lat,...,scan_direction_strength_k4,solution_id,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,vbroad,vbroad_error,vbroad_nb_transits,visibility_periods_used,other_ids
0,"210.6310833,9.6868889",0.001331,3723554268436602240,Gaia DR3 3723554268436602240,210.631835,9.685783,350.165278,65.804284,204.909417,20.741221,...,NaN,1636148068921376768,6028.7090,6002.4976,6157.5840,NaN,NaN,NaN,15,NaN
1,"254.4646667,-3.5546389",0.001120,4377687545785813248,Gaia DR3 4377687545785813248,254.464316,-3.555703,15.741641,23.212315,253.576810,19.002220,...,NaN,1636148068921376768,4833.0923,4805.4240,4942.8110,12.950286,9.241796,17.0,20,"Gaia DR2 4377687545785813248, Gaia E/DR1 43776..."
2,"241.7759167,-20.8308611",0.002537,6243816407441434752,Gaia DR3 6243816407441434752,241.775862,-20.833398,352.671953,22.609824,243.768326,0.072409,...,NaN,1636148068921376768,4774.2640,4680.0440,4792.1304,NaN,NaN,NaN,15,"Gaia DR2 6243816407441434752, Gaia E/DR1 62438..."
3,"288.6929583,-22.1933611",0.000549,4081038754562022528,Gaia DR3 4081038754562022528,288.692649,-22.193830,15.270882,-14.832483,287.262366,0.132229,...,0.885026,1636148068921376768,NaN,NaN,NaN,NaN,NaN,NaN,17,"Gaia DR2 4081038754562022528, Gaia E/DR1 40810..."
4,"275.0843333,-32.3048611",0.000569,4045622011959560832,Gaia DR3 4045622011959560832,275.084719,-32.304394,0.712755,-8.159175,274.349064,-8.941845,...,0.638802,1636148068921376768,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN
5,"320.0565417,-33.9008056",0.000186,6782821815106683008,Gaia DR3 6782821815106683008,320.056319,-33.900831,10.896924,-44.500011,311.837289,-17.443215,...,NaN,1636148068921376768,NaN,NaN,NaN,NaN,NaN,NaN,16,"Gaia DR2 6782821815106683008, Gaia E/DR1 67828..."
6,"86.4687083,-25.9449167",0.000923,2914192523598690048,Gaia DR3 2914192523598690048,86.468840,-25.944001,230.531992,-25.170987,85.125545,-49.323550,...,NaN,1636148068921376768,5434.1753,5402.4520,5440.1450,NaN,NaN,NaN,27,"Gaia DR2 2914192523598690048, Gaia E/DR1 29141..."
7,"351.8465,4.4472222",0.006673,2659914516754259456,Gaia DR3 2659914516754259456,351.852262,4.443828,87.010712,-52.467434,354.282134,7.313242,...,0.675912,1636148068921376768,6123.6074,6101.1284,6142.4385,NaN,NaN,NaN,13,"Gaia DR2 2659914516754259456, Gaia E/DR1 26599..."
8,"6.3159583,16.0463611",0.005386,2792504722733373312,Gaia DR3 2792504722733373312,6.314779,16.051627,113.800247,-46.355437,12.225750,12.218525,...,0.216838,1636148068921376768,NaN,NaN,NaN,NaN,NaN,NaN,17,"Gaia DR2 2792504722733373312, Gaia E/DR1 27925..."
9,"339.857625,15.8131667",0.002821,2733469542550276352,Gaia DR3 2733469542550276352,339.854847,15.812263,82.284061,-36.432515,347.779710,22.445934,...,0.765033,1636148068921376768,5965.4880,5880.6670,6038.9880,NaN,NaN,NaN,13,"Gaia DR2 2733469542550276352, Gaia E/DR1 27334..."
